In [ ]:
import shap
import pandas as pd
import numpy as np
import xgboost as xgb
import lightgbm as lgb
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold, RandomizedSearchCV, cross_val_predict
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report, make_scorer, average_precision_score
import matplotlib.pyplot as plt
import seaborn as sns
from imblearn.pipeline import Pipeline as ImbPipeline
from imblearn.over_sampling import SMOTE
from sklearn.linear_model import LinearRegression
from itertools import combinations
from scipy.stats import ttest_ind
from sklearn.inspection import permutation_importance
from sklearn import tree
from sklearn.experimental import enable_iterative_imputer  # Required to enable
from sklearn.impute import IterativeImputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import roc_auc_score



# Load and preprocess data
file_path = r"C:\\Users\\SANDY\\Downloads\\carbon dots.csv"
df = pd.read_csv(file_path, encoding='latin1')

df.columns = [
    'PlantName', 'Part', 'Family', 'Solvent', 'ParticleSize', 'ZetaPotential',
    'CellType', 'CellOrigin', 'CellLine', 'Assay', 'Dose', 'Viability', 'Time'
]

num_cols = ['ParticleSize', 'ZetaPotential', 'Dose', 'Time']
df[num_cols] = df[num_cols].apply(pd.to_numeric, errors='coerce')

# Improved RandomForest for better imputation accuracy
rf_estimator = RandomForestRegressor(
    n_estimators=100,
    max_depth=6,
    min_samples_split=5,
    max_features='sqrt',
    random_state=42,
    n_jobs=-1
)

imputer = IterativeImputer(
    estimator=rf_estimator,
    max_iter=10,
    random_state=42
)

# Apply imputation
df[num_cols] = imputer.fit_transform(df[num_cols])

# Drop rows with any missing values
df.dropna(inplace=True)

# Create target column for classification
df['Toxicity'] = (df['Viability'] < 50).astype(int)

from sklearn.preprocessing import LabelEncoder

# Remove target & leakage column
df_lgb = df.drop(columns=['Viability', 'Toxicity']).copy()
df_lgb['Toxicity'] = df['Toxicity']

categorical_cols_lgb = ['Family', 'Solvent', 'CellType', 'CellOrigin']
label_encoders = {}

for col in categorical_cols_lgb:
    df_lgb[col] = df_lgb[col].astype("category")

    


# Keep full dataframe for raw_df (for plotting)
raw_df = df.copy()

# Prepare X with only the 8 features mentioned in the project paper
X = df[['Family', 'Solvent', 'ParticleSize', 'ZetaPotential', 'CellType', 'CellOrigin', 'Dose', 'Time']]

# Separate features with and without Family
X_family = X.copy()
X_no_family = X.drop(columns=['Family'])
y = df['Toxicity']

categorical_cols = ['Family', 'Solvent', 'CellType', 'CellOrigin']
categorical_cols_no_family = ['Solvent', 'CellType', 'CellOrigin']

X_lgb_family = df_lgb[['Family', 'Solvent', 'CellType', 'CellOrigin', 'ParticleSize', 'ZetaPotential', 'Dose', 'Time']]
X_lgb_no_family = X_lgb_family.drop(columns=['Family'])

categorical_features_family = [X_lgb_family.columns.get_loc(col) for col in categorical_cols_lgb]
categorical_features_no_family = [X_lgb_no_family.columns.get_loc(col) for col in categorical_cols_lgb if col != 'Family']

# ColumnTransformer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

preprocessor_with_family = ColumnTransformer(transformers=[
    ('num', StandardScaler(), num_cols),
    ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols)
])

preprocessor_no_family = ColumnTransformer(transformers=[
    ('num', StandardScaler(), num_cols),
    ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols_no_family)
])

cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
f1_scorer = make_scorer(f1_score)

# Hyperparameters
param_grids = {
    'XGBoost': {'clf__n_estimators': [50, 100, 200], 'clf__max_depth': [3, 5, 10], 
                'clf__learning_rate': [0.01, 0.1, 0.2], 'clf__subsample': [0.8, 1.0], 'clf__colsample_bytree': [0.8, 1.0]},
    'AdaBoost': {'clf__n_estimators': [50, 100, 200], 'clf__learning_rate': [0.01, 0.1, 0.2]},
    'LightGBM': {
        "n_estimators": [100, 200],
        "learning_rate": [0.01, 0.1],
        "max_depth": [3, 5],
        "colsample_bytree": [0.6, 0.8],
        "subsample": [0.8, 1.0]
    },
    'DecisionTree': {
    'clf__max_depth': [3, 5, 10, None],
    'clf__min_samples_split': [2, 5, 10],
    'clf__min_samples_leaf': [1, 2, 4]
    },
    'RandomForest': {
    'clf__n_estimators': [50, 100, 200],
    'clf__max_depth': [3, 5, 10, None],
    'clf__min_samples_split': [2, 5, 10],
    'clf__max_features': ['sqrt', 'log2']
    },
    'GradientBoosting': {
    'clf__n_estimators': [50, 100, 200],
    'clf__learning_rate': [0.01, 0.1, 0.2],
    'clf__max_depth': [3, 5, 10]
    }

}

models = {
    'XGBoost': xgb.XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss'),
    'AdaBoost': AdaBoostClassifier(DecisionTreeClassifier(max_depth=1), random_state=42),
    'LightGBM': lgb.LGBMClassifier(random_state=42),
    'DecisionTree': DecisionTreeClassifier(random_state=42),
    'RandomForest': RandomForestClassifier(random_state=42),
    'GradientBoosting': GradientBoostingClassifier(random_state=42)

    
}

conf_matrices = {}

# Evaluation function
def evaluate_model(name, model, param_grid, X, y, dataset_name, preprocessor=None, categorical_features=None):
    print(f"Training {name} on {dataset_name}...")

    if name == "LightGBM":
        # Ensure categorical columns are label encoded before SMOTE
        X_encoded = X.copy()
        # Map indices to column names
        categorical_colnames = [X.columns[i] if isinstance(i, int) else i for i in categorical_features]

        for col in categorical_colnames:
            le = LabelEncoder()
            X_encoded[col] = le.fit_transform(X_encoded[col])


        smote = SMOTE(sampling_strategy=0.6, random_state=42)
        X_resampled, y_resampled = smote.fit_resample(X_encoded, y)


        random_search = RandomizedSearchCV(
        model, param_distributions=param_grid, n_iter=20,
        scoring=f1_scorer, cv=cv, verbose=1, n_jobs=-1, random_state=42
    )
        random_search.fit(X_resampled, y_resampled, categorical_feature=categorical_features)

        best_model = random_search.best_estimator_

    else:
        # Define pipeline first
        pipeline = ImbPipeline([
            ('preprocessor', preprocessor),
            ('smote', SMOTE(sampling_strategy=0.6, random_state=42)),
            ('clf', model)
        ])

        random_search = RandomizedSearchCV(
            pipeline, param_distributions=param_grid, n_iter=20,
            scoring=f1_scorer, cv=cv, verbose=1, n_jobs=-1, random_state=42
        )
        random_search.fit(X, y)
        best_model = random_search.best_estimator_

    # Evaluate
    y_pred = cross_val_predict(best_model, X, y, cv=cv)
    cm = confusion_matrix(y, y_pred)
    conf_matrices[f"{name} ({dataset_name})"] = cm

    acc = accuracy_score(y, y_pred)
    prec = precision_score(y, y_pred)
    rec = recall_score(y, y_pred)
    f1_val = f1_score(y, y_pred)
    roc_auc = roc_auc_score(y, y_pred)

    with open("model_text_results.txt", "a") as f:
        f.write(f"\n=== {name} ({dataset_name}) ===\n")
        f.write(f"Accuracy: {acc:.3f}\n")
        f.write(f"Precision: {prec:.3f}\n")
        f.write(f"Recall: {rec:.3f}\n")
        f.write(f"F1 Score: {f1_val:.3f}\n")
        f.write(f"ROC-AUC: {roc_auc:.3f}\n")
        f.write(f"Classification Report:\n{classification_report(y, y_pred)}\n")

    return best_model

print("\n### TRAINING MODELS ###")
best_models = {}
for dataset_name, (X, preprocessor, cat_features) in {
    "With Family": (X_family, preprocessor_with_family, None),
    "Without Family": (X_no_family, preprocessor_no_family, None),
    "LGBM With Family": (X_lgb_family, None, categorical_features_family),
    "LGBM Without Family": (X_lgb_no_family, None, categorical_features_no_family),
}.items():
    for name in models:
        if "LGBM" in dataset_name and name != "LightGBM":
            continue  # Skip non-LightGBM for LGBM input
        elif "LGBM" not in dataset_name and name == "LightGBM":
            continue  # Skip LightGBM in OneHot datasets

        best_models[f"{name} ({dataset_name})"] = evaluate_model(
            name, models[name], param_grids[name], X, y, dataset_name,
            preprocessor=preprocessor, categorical_features=cat_features
        )




# Confusion Matrices
for name, cm in conf_matrices.items():
    plt.figure(figsize=(6, 5))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.title(f"Confusion Matrix - {name}")
    plt.ylabel("True Label")
    plt.xlabel("Predicted Label")
    plt.tight_layout()
    plt.savefig(f"confusion_matrix_{name.replace(' ', '_')}.png")
    plt.close()

# # --- SHAP + Boxplots + Confusion Matrix (updated for GradientBoosting too) ---
# for dataset_name, X in [("With Family", X_family), ("Without Family", X_no_family)]:
#     for name, model in best_models.items():
#         if dataset_name in name:
#             clf = model.named_steps["clf"]
#             y_pred = cross_val_predict(model, X, y, cv=cv)
#             cm = confusion_matrix(y, y_pred)

#             # Save confusion matrix
#             plt.figure(figsize=(6, 5))
#             sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
#             plt.title(f"Confusion Matrix - {name}")
#             plt.ylabel("True Label")
#             plt.xlabel("Predicted Label")
#             plt.tight_layout()
#             plt.savefig(f"confusion_matrix_{name.replace(' ', '_')}.png")
#             plt.close()

#             # SHAP Explainer
#             try:
#                 if "AdaBoost" in name:
#                     explainer = shap.KernelExplainer(clf.predict, shap.sample(X, 50))
#                     shap_values = explainer.shap_values(X)
#                 else:
#                     explainer = shap.TreeExplainer(clf)
#                     shap_values = explainer.shap_values(X)
#             except Exception as e:
#                 print(f"SHAP failed for {name}: {e}")
#                 continue

#             # SHAP summary plot
#             plt.figure(figsize=(10, 6))
#             shap.summary_plot(shap_values, X, feature_names=X.columns, show=False)
#             plt.savefig(f"shap_summary_{name.replace(' ', '_')}.png")
#             plt.close()

#             # SHAP boxplot
#             # SHAP boxplot (safe for binary classifiers including DecisionTree and RandomForest)
#             try:
#                 if isinstance(shap_values, list):
#                     shap_array = shap_values[1]  # multiclass shap output from KernelExplainer
#                 elif isinstance(shap_values, np.ndarray) and shap_values.ndim == 3:
#                     shap_array = shap_values[:, :, 1]  # pick class 1 for binary output
#                 else:
#                     shap_array = shap_values

#                 shap_df = pd.DataFrame(shap_array, columns=X.columns)
#                 plt.figure(figsize=(10, 6))
#                 sns.boxplot(data=shap_df)
#                 plt.xticks(rotation=90)
#                 plt.title(f"Boxplot of Feature Importance - {name}")
#                 plt.ylabel("SHAP Value")
#                 plt.tight_layout()
#                 plt.savefig(f"boxplot_shap_{name.replace(' ', '_')}.png")
#                 plt.close()
#             except Exception as e:
#                 print(f"SHAP Boxplot failed for {name}: {e}")



# # IC50-like Boxplot
# df_raw = pd.read_csv(file_path, encoding='latin1')
# df_raw.columns = [
#     'PlantName', 'Part', 'Family', 'Solvent', 'ParticleSize', 'ZetaPotential',
#     'CellType', 'CellOrigin', 'CellLine', 'Assay', 'Dose', 'Viability', 'Time'
# ]
# df_raw['Dose'] = pd.to_numeric(df_raw['Dose'], errors='coerce')
# df_raw.dropna(subset=['Dose', 'Family'], inplace=True)

# plt.figure(figsize=(12, 6))
# sns.boxplot(x='Family', y='Dose', data=df_raw)
# plt.xticks(rotation=45)
# plt.title("Approximate IC50-like Boxplot (based on Dose vs Family)")
# plt.ylabel("Dose (mg/L) -- approximate IC50 representation")
# plt.xlabel("Family")
# plt.tight_layout()
# plt.savefig("approximate_ic50_boxplot.png")
# plt.close()

# # P-VALUE Heatmap
# families = df_raw['Family'].unique()
# p_matrix = pd.DataFrame(np.ones((len(families), len(families))), index=families, columns=families)

# for fam1, fam2 in combinations(families, 2):
#     doses1 = df_raw[df_raw['Family'] == fam1]['Dose']
#     doses2 = df_raw[df_raw['Family'] == fam2]['Dose']
#     stat, p_value = ttest_ind(doses1, doses2)
#     p_matrix.loc[fam1, fam2] = p_value
#     p_matrix.loc[fam2, fam1] = p_value

# cmap = sns.color_palette("Reds", as_cmap=True)
# plt.figure(figsize=(12, 8))
# sns.heatmap(p_matrix, annot=False, cmap=cmap, cbar_kws={'label': 'P value'}, vmin=0, vmax=1)
# plt.title("Pairwise T-Test P-Value Heatmap (Dose between Families)")
# plt.xticks(rotation=45)
# plt.tight_layout()
# plt.savefig("pvalue_heatmap_families.png")
# plt.close()

# # RANDOM FOREST Importance
# X_family_encoded = preprocessor_with_family.fit_transform(X_family)
# rf_with_family = RandomForestClassifier(random_state=42)
# rf_with_family.fit(X_family_encoded, y)
# importances_gini = rf_with_family.feature_importances_
# perm_importance = permutation_importance(rf_with_family, X_family_encoded, y, n_repeats=10, random_state=42, scoring='accuracy')
# importances_accuracy = perm_importance.importances_mean

# feat_importance = pd.DataFrame({
#     'Feature': preprocessor_with_family.get_feature_names_out(),
#     'MeanDecreaseGini': importances_gini,
#     'MeanDecreaseAccuracy': importances_accuracy
# }).sort_values('MeanDecreaseAccuracy', ascending=False)

# plt.figure(figsize=(8, 6))
# sns.barplot(x='MeanDecreaseAccuracy', y='Feature', data=feat_importance)
# plt.title("Mean Decrease Accuracy (Random Forest)")
# plt.tight_layout()
# plt.savefig("rf_mean_decrease_accuracy.png")
# plt.close()

# plt.figure(figsize=(8, 6))
# sns.barplot(x='MeanDecreaseGini', y='Feature', data=feat_importance)
# plt.title("Mean Decrease Gini (Random Forest)")
# plt.tight_layout()
# plt.savefig("rf_mean_decrease_gini.png")
# plt.close()
from sklearn.inspection import permutation_importance

def compute_and_plot_importances(model, model_name, X, y, feature_names, gini_applicable=False):
    print(f"Calculating importances for: {model_name}")

    # Compute permutation importance (Mean Decrease Accuracy)
    result = permutation_importance(model, X, y, n_repeats=10, random_state=42, scoring='accuracy')
    importance_df = pd.DataFrame({
        'Feature': feature_names,
        'MeanDecreaseAccuracy': result.importances_mean
    })

    # If Gini applicable (RF, DT)
    if gini_applicable and hasattr(model, 'feature_importances_'):
        importance_df['MeanDecreaseGini'] = model.feature_importances_

    # Plot Mean Decrease Accuracy
    plt.figure(figsize=(8, 6))
    sns.barplot(x='MeanDecreaseAccuracy', y='Feature', data=importance_df.sort_values('MeanDecreaseAccuracy', ascending=False))
    plt.title(f"{model_name} - Mean Decrease Accuracy")
    plt.tight_layout()
    plt.savefig(f"importance_accuracy_{model_name.replace(' ', '_')}.png")
    plt.close()

    # Plot Mean Decrease Gini if available
    if 'MeanDecreaseGini' in importance_df.columns:
        plt.figure(figsize=(8, 6))
        sns.barplot(x='MeanDecreaseGini', y='Feature', data=importance_df.sort_values('MeanDecreaseGini', ascending=False))
        plt.title(f"{model_name} - Mean Decrease Gini")
        plt.tight_layout()
        plt.savefig(f"importance_gini_{model_name.replace(' ', '_')}.png")
        plt.close()

for name, model in best_models.items():
    # Skip LightGBM — not compatible with sklearn's permutation_importance due to categorical_feature mismatch
    if "LightGBM" in name:
        print(f"Skipping permutation importance for {name} (LightGBM)")
        continue

    if hasattr(model, 'named_steps'):
        clf = model.named_steps['clf']
        preprocessor = model.named_steps['preprocessor']
        X_transformed = preprocessor.transform(X_family) if "With Family" in name else preprocessor.transform(X_no_family)
        feature_names = preprocessor.get_feature_names_out()
    else:
        clf = model
        X_transformed = X_lgb_family if "With Family" in name else X_lgb_no_family
        feature_names = X_transformed.columns

    gini_flag = isinstance(clf, (RandomForestClassifier, DecisionTreeClassifier))

    compute_and_plot_importances(
        clf,
        model_name=name,
        X=X_transformed,
        y=y,
        feature_names=feature_names,
        gini_applicable=gini_flag
    )






# Plot decision tree or random forest from already trained best models
from lightgbm import plot_tree as lgb_plot_tree
import xgboost as xgb

# for name, model in best_models.items():
#     clf = model.named_steps['clf']
#     dataset_name = "With Family" if "With Family" in name else "Without Family"
#     X = X_family if "With Family" in name else X_no_family

#     # Decision Tree
#     if isinstance(clf, DecisionTreeClassifier):
#         plt.figure(figsize=(20, 10))
#         tree.plot_tree(clf, feature_names=X.columns, filled=True)
#         plt.title(f"Decision Tree - {name}")
#         plt.savefig(f"decision_tree_{name.replace(' ', '_')}.png")
#         plt.close()

#     # Random Forest, Gradient Boosting, AdaBoost
#     elif hasattr(clf, 'estimators_'):
#         estimator_type = type(clf).__name__
#         try:
#             if isinstance(clf, GradientBoostingClassifier):
#                 estimator = clf.estimators_[0, 0]  # fix for 2D array
#             else:
#                 estimator = clf.estimators_[0]  # e.g. RandomForest, AdaBoost

#             plt.figure(figsize=(20, 10))
#             tree.plot_tree(estimator, feature_names=X.columns, filled=True)
#             plt.title(f"{estimator_type} (First Tree) - {name}")
#             plt.savefig(f"{estimator_type.lower()}_tree_{name.replace(' ', '_')}.png")
#             plt.close()
#         except Exception as e:
#             print(f"Plotting failed for {name}: {e}")


#     # XGBoost
#     elif isinstance(clf, xgb.XGBClassifier):
#         try:
#             plt.figure(figsize=(20, 10))
#             xgb.plot_tree(clf, num_trees=0)
#             plt.title(f"XGBoost Tree - {name}")
#             plt.savefig(f"xgboost_tree_{name.replace(' ', '_')}.png")
#             plt.close()
#         except Exception as e:
#             print(f"XGBoost tree plotting failed for {name}: {e}")

#     # LightGBM
#     elif isinstance(clf, lgb.LGBMClassifier):
#         try:
#             ax = lgb_plot_tree(clf, tree_index=0, figsize=(20, 10), show_info=['split_gain', 'internal_value', 'leaf_count'])
#             plt.title(f"LightGBM Tree - {name}")
#             plt.savefig(f"lightgbm_tree_{name.replace(' ', '_')}.png")
#             plt.close()
#         except Exception as e:
#             print(f"LightGBM tree plotting failed for {name}: {e}")

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from itertools import combinations
from scipy.stats import ttest_ind

# Assume df_raw with columns: ['Family', 'Dose'] is already prepared

# Step 1: Compute p-value matrix
families = sorted(raw_df['Family'].unique())
p_matrix = pd.DataFrame(index=families, columns=families, dtype=float)

for fam1, fam2 in combinations(families, 2):
    group1 = raw_df[raw_df['Family'] == fam1]['Dose']
    group2 = raw_df[raw_df['Family'] == fam2]['Dose']
    stat, p = ttest_ind(group1, group2, equal_var=False)
    p_matrix.loc[fam1, fam2] = p
    p_matrix.loc[fam2, fam1] = p

np.fill_diagonal(p_matrix.values, np.nan)

# Step 2: Categorize p-values for coloring
def categorize_pvalue(p):
    if pd.isna(p):
        return np.nan
    elif p < 0.01:
        return 0.01
    elif p < 0.05:
        return 0.05
    else:
        return 1

color_matrix = p_matrix.map(categorize_pvalue)

# Optional: Abbreviate family names
abbrev_families = {fam: fam[:4] + '.' if len(fam) > 4 else fam for fam in families}
color_matrix.rename(index=abbrev_families, columns=abbrev_families, inplace=True)
p_matrix.rename(index=abbrev_families, columns=abbrev_families, inplace=True)

# Step 3: Plotting
plt.figure(figsize=(8, 6))
sns.heatmap(color_matrix, cmap=["#d73027", "#fc8d59", "#ffffbf"],  # deep red, orange, white
            cbar_kws={'label': 'p value'}, linewidths=0.5, square=True, linecolor='gray')

# Add star annotations
for i in range(len(color_matrix)):
    for j in range(len(color_matrix)):
        pval = p_matrix.iloc[i, j]
        if pd.notna(pval):
            if pval < 0.01:
                plt.text(j + 0.5, i + 0.5, "**", ha='center', va='center', color='black', fontsize=10)
            elif pval < 0.05:
                plt.text(j + 0.5, i + 0.5, "*", ha='center', va='center', color='black', fontsize=10)

plt.xticks(rotation=45, ha='right')
plt.title("P-value Matrix (Dose vs Family)", fontsize=13)
plt.tight_layout()
plt.savefig("pvalue_discrete_star_heatmap.png")
plt.close()

from sklearn.tree import export_graphviz
from graphviz import Source
import os
from sklearn.tree import _tree
from graphviz import Digraph
def custom_tree_to_graphviz(clf, feature_names, filename='custom_tree'):
    tree_ = clf.tree_
    dot = Digraph(name="CustomTree", format='png')
    dot.attr('node', shape='ellipse', style='filled', fontname='Helvetica', fontsize='12')

    def add_node(node_id):
        if tree_.feature[node_id] != _tree.TREE_UNDEFINED:
            name = feature_names[tree_.feature[node_id]]
            threshold = round(tree_.threshold[node_id], 3)
            label = f"{name} ≤ {threshold}"
            dot.node(str(node_id), label=label, fillcolor="white")
        else:
            value = tree_.value[node_id][0]
            pred_class = int(value.argmax())
            label = "Toxic" if pred_class == 1 else "Non-Toxic"
            color = "red" if label == "Toxic" else "green"
            dot.node(str(node_id), label=label, fillcolor=color, shape='box', fontcolor="white")

    def recurse(node_id):
        add_node(node_id)
        if tree_.feature[node_id] != _tree.TREE_UNDEFINED:
            left_child = tree_.children_left[node_id]
            right_child = tree_.children_right[node_id]
            recurse(left_child)
            recurse(right_child)
            dot.edge(str(node_id), str(left_child), label="yes")
            dot.edge(str(node_id), str(right_child), label="no")

    recurse(0)
    dot.render(filename, cleanup=True)
    print(f"Custom tree saved to: {filename}.png")


# Ensure output folder exists
os.makedirs("graphviz_trees", exist_ok=True)

for name, model in best_models.items():
    # If model is a pipeline, extract clf
    if hasattr(model, 'named_steps'):
        clf = model.named_steps['clf']
    else:
        clf = model  # For LightGBM, XGBoost etc., directly use the model

    try:
        # Handle feature names
        if "LightGBM" in name:
            if "Without Family" in name:
                feature_names = X_lgb_no_family.columns.tolist()
            else:
                feature_names = X_lgb_family.columns.tolist()

        else:
            preprocessor = model.named_steps['preprocessor']
            feature_names = preprocessor.get_feature_names_out()

        # --- Tree-based models ---
        if isinstance(clf, DecisionTreeClassifier):
            dt = clf
        elif isinstance(clf, RandomForestClassifier):
            dt = clf.estimators_[0]
        elif isinstance(clf, AdaBoostClassifier):
            dt = clf.estimators_[0]
        elif isinstance(clf, GradientBoostingClassifier):
            dt = clf.estimators_[0, 0]
        else:
            dt = None

        if dt is not None:
            # Standard Graphviz Export
            dot_data = export_graphviz(
                dt,
                out_file=None,
                feature_names=feature_names,
                class_names=['Non-Toxic', 'Toxic'],
                filled=True,
                rounded=True,
                special_characters=True
            )
            graph = Source(dot_data)
            graph.render(f"graphviz_trees/standard_tree_{name.replace(' ', '_')}", format="png", cleanup=True)

            # Custom Graphviz Export
            custom_tree_to_graphviz(
                dt,
                feature_names=feature_names,
                filename=f"graphviz_trees/custom_tree_{name.replace(' ', '_')}"
            )

        # --- XGBoost ---
        elif hasattr(clf, "get_booster"):
            booster = clf.get_booster()
            booster.feature_names = list(map(str, feature_names))  # Ensure strings
            fig, ax = plt.subplots(figsize=(30, 20), dpi=150)
            xgb.plot_tree(booster, num_trees=0, ax=ax)
            plt.title(f"XGBoost Tree 0 - {name}", fontsize=20)
            plt.tight_layout()
            plt.savefig(f"graphviz_trees/xgboost_tree_{name.replace(' ', '_')}_tree0.png", dpi=300)
            plt.close()

        # --- LightGBM ---
        elif isinstance(clf, lgb.LGBMClassifier):
            ax = lgb.plot_tree(clf, tree_index=0, figsize=(20, 10),
                               show_info=['split_gain', 'internal_value', 'leaf_count'])
            plt.title(f"LightGBM Tree 0 - {name}")
            plt.tight_layout()
            plt.savefig(f"graphviz_trees/lightgbm_tree_{name.replace(' ', '_')}.png")
            plt.close()

    except Exception as e:
        print(f"Custom Graphviz export failed for {name}: {e}")


import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from itertools import combinations
from scipy.stats import ttest_rel

# Reload or use already processed df_raw
families = sorted(raw_df['Family'].unique())
p_matrix = pd.DataFrame(index=families, columns=families, dtype=float)

# # Try paired t-test (ttest_rel)
# for fam1, fam2 in combinations(families, 2):
#     group1 = raw_df[raw_df['Family'] == fam1]['Dose'].dropna().values
#     group2 = raw_df[raw_df['Family'] == fam2]['Dose'].dropna().values

#     min_len = min(len(group1), len(group2))

#     if min_len == 0:
#         p = np.nan
#     else:
#         # Force same length by truncation (not statistically correct)
#         g1 = group1[:min_len]
#         g2 = group2[:min_len]

#         try:
#             stat, p = ttest_rel(g1, g2)
#         except Exception as e:
#             print(f"Error with {fam1} vs {fam2}: {e}")
#             p = np.nan

#     p_matrix.loc[fam1, fam2] = p
#     p_matrix.loc[fam2, fam1] = p

# # Fill diagonal
# np.fill_diagonal(p_matrix.values, np.nan)

# # Categorize p-values
# def categorize_p(p):
#     if pd.isna(p):
#         return np.nan
#     elif p < 0.01:
#         return 0.01
#     elif p < 0.05:
#         return 0.05
#     else:
#         return 1

# color_matrix = p_matrix.map(categorize_p)

# # Optional: shorten names
# abbrev = {fam: fam[:4] + '.' if len(fam) > 4 else fam for fam in families}
# color_matrix.rename(index=abbrev, columns=abbrev, inplace=True)
# p_matrix.rename(index=abbrev, columns=abbrev, inplace=True)

# # Plot heatmap
# plt.figure(figsize=(8, 6))
# sns.heatmap(color_matrix, cmap=["#d73027", "#fc8d59", "#ffffbf"],  # red, orange, white
#             cbar_kws={'label': 'p value'}, linewidths=0.5, square=True, linecolor='gray')

# # Annotate with stars
# for i in range(len(color_matrix)):
#     for j in range(len(color_matrix)):
#         val = p_matrix.iloc[i, j]
#         if pd.notna(val):
#             if val < 0.01:
#             elif val < 0.05:
#                 plt.text(j + 0.5, i + 0.5, "*", ha='center', va='center', color='black', fontsize=10)

# plt.title("⚠️ Paired t-test Matrix (Incorrect Assumption)", fontsize=13)
# plt.xticks(rotation=45)
# plt.tight_layout()
# plt.savefig("paired_ttest_pvalue_matrix.png")
# plt.show()

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.optimize import curve_fit

# --- Define sigmoid ---
def sigmoid(x, bottom, top, logIC50, hill_slope):
    return bottom + (top - bottom) / (1 + 10**((logIC50 - np.log10(x)) * hill_slope))

# --- Estimate IC50s using raw_df ---
ic50_data = []
for (plant, cell_line), group in raw_df.groupby(['PlantName', 'CellLine']):
    dose = group['Dose'].values
    viability = group['Viability'].values
    if len(dose) < 3:
        continue
    try:
        p0 = [0, 100, np.log10(np.median(dose)), 1]
        params, _ = curve_fit(sigmoid, dose, viability, p0, maxfev=5000)
        ic50 = 10**params[2]
        family = group['Family'].iloc[0]
        ic50_data.append({'Plant': plant, 'CellLine': cell_line, 'Family': family, 'IC50': ic50})
    except:
        continue

ic50_df = pd.DataFrame(ic50_data)

# --- Filter stable/reliable families ---
reliable_families = [
    'Malvaceae', 'Brassicaceae', 'Oleaceae',
    'Juglandaceae', 'Rosaceae', 'Rutaceae', 'Convolvulaceae'
]
filtered_df = ic50_df[ic50_df['Family'].isin(reliable_families)].copy()

# --- Add counts to labels ---
counts = filtered_df['Family'].value_counts()
filtered_df['FamilyLabel'] = filtered_df['Family'].apply(lambda f: f"{f} (n={counts[f]})")

# --- Plot and Save as PNG ---
plt.figure(figsize=(14, 6))
sns.boxplot(data=filtered_df, x='FamilyLabel', y='IC50', palette='Set3')
plt.title('IC₅₀ by Plant Family (Cleaned, Reliable Subset)')
plt.ylabel('IC₅₀ (mg/L)')
plt.xlabel('Plant Family')
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig("graphviz_trees/ic50_boxplot.png", dpi=300)
plt.close()








### TRAINING MODELS ###
Training XGBoost on With Family...
Fitting 10 folds for each of 20 candidates, totalling 200 fits


c:\Users\SANDY\AppData\Local\Programs\Python\Python312\Lib\site-packages\xgboost\training.py:183: UserWarning: [18:53:07] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
c:\Users\SANDY\AppData\Local\Programs\Python\Python312\Lib\site-packages\xgboost\training.py:183: UserWarning: [18:53:07] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
c:\Users\SANDY\AppData\Local\Programs\Python\Python312\Lib\site-packages\xgboost\training.py:183: UserWarning: [18:53:07] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
c:\Users\SANDY\AppData\Local\Programs\Python\Python312\Lib\site-packages\xgboost\training.py:183: UserWarning: [18:53:07] WARNING: C:\actio

Training AdaBoost on With Family...
Fitting 10 folds for each of 9 candidates, totalling 90 fits


c:\Users\SANDY\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_search.py:317: UserWarning: The total space of parameters 9 is smaller than n_iter=20. Running 9 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Training DecisionTree on With Family...
Fitting 10 folds for each of 20 candidates, totalling 200 fits
Training RandomForest on With Family...
Fitting 10 folds for each of 20 candidates, totalling 200 fits
Training GradientBoosting on With Family...
Fitting 10 folds for each of 20 candidates, totalling 200 fits
Training XGBoost on Without Family...
Fitting 10 folds for each of 20 candidates, totalling 200 fits


...

c:\Users\SANDY\AppData\Local\Programs\Python\Python312\Lib\site-packages\xgboost\plotting.py:267: FutureWarning: The `num_trees` parameter is deprecated, use `tree_idx` insetad. 
  warnings.warn(


Custom tree saved to: graphviz_trees/custom_tree_AdaBoost_(With_Family).png
Custom tree saved to: graphviz_trees/custom_tree_DecisionTree_(With_Family).png
Custom tree saved to: graphviz_trees/custom_tree_RandomForest_(With_Family).png
Custom tree saved to: graphviz_trees/custom_tree_GradientBoosting_(With_Family).png


c:\Users\SANDY\AppData\Local\Programs\Python\Python312\Lib\site-packages\xgboost\plotting.py:267: FutureWarning: The `num_trees` parameter is deprecated, use `tree_idx` insetad. 
  warnings.warn(


Custom tree saved to: graphviz_trees/custom_tree_AdaBoost_(Without_Family).png
Custom tree saved to: graphviz_trees/custom_tree_DecisionTree_(Without_Family).png
Custom tree saved to: graphviz_trees/custom_tree_RandomForest_(Without_Family).png
Custom tree saved to: graphviz_trees/custom_tree_GradientBoosting_(Without_Family).png


C:\Users\SANDY\AppData\Local\Temp\ipykernel_14432\1245031682.py:781: OptimizeWarning: Covariance of the parameters could not be estimated
  params, _ = curve_fit(sigmoid, dose, viability, p0, maxfev=5000)
C:\Users\SANDY\AppData\Local\Temp\ipykernel_14432\1245031682.py:770: RuntimeWarning: divide by zero encountered in log10
  return bottom + (top - bottom) / (1 + 10**((logIC50 - np.log10(x)) * hill_slope))
C:\Users\SANDY\AppData\Local\Temp\ipykernel_14432\1245031682.py:803: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.boxplot(data=filtered_df, x='FamilyLabel', y='IC50', palette='Set3')
